In [23]:
import numpy as np
import pandas as pd
import json

In [38]:
data = pd.read_csv('WDIData.csv')
data

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
0,Arab World,ARB,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,ARB,"2005 PPP conversion factor, private consumptio...",PA.NUS.PRVT.PP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,8.347462e+01,8.400608e+01,8.441615e+01,8.473457e+01,8.500364e+01,8.524497e+01,NaN,NaN,NaN,NaN
3,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,8.465534e+01,8.595535e+01,8.639231e+01,8.684697e+01,8.760496e+01,8.803912e+01,NaN,NaN,NaN,NaN
4,Arab World,ARB,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,7.176534e+01,7.373427e+01,7.450072e+01,7.521850e+01,7.671225e+01,7.740727e+01,NaN,NaN,NaN,NaN
5,Arab World,ARB,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,9.472336e+01,9.529989e+01,9.539420e+01,9.547506e+01,9.566886e+01,9.608860e+01,NaN,NaN,NaN,NaN
6,Arab World,ARB,Account (% age 15+) [ts],WP_time_10.1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Arab World,ARB,Account at a financial institution (% age 15+),WP_time_01.1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Arab World,ARB,"Account at a financial institution, female (% ...",WP_time_01.3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Arab World,ARB,"Account at a financial institution, income, po...",WP_time_01.8,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Write unique countries for evaluation
countries = data['Country Name'].unique().tolist()
with open('country_names.txt','w') as outfile:
    json.dump(countries, outfile, indent=2)
    
# Write unique indicators for evaluation
indicators = data['Indicator Name'].unique().tolist()
with open('indicator_names.txt', 'w') as outfile:
    json.dump(indicators, outfile, indent=2)

In [36]:
# Start with geographic regions to simplify this excercise
geo_list = ['Arab World', 'Caribbean small states', 'East Asia & Pacific', 
            'Euro area', 'Europe & Central Asia', 'European Union', 
            'Latin America & Caribbean', 'Middle East & North Africa', 
            'North America', 'Pacific island small states', 'South Asia', 
           'Sub-Saharan Africa']

# Restrict indicators to evaluate
ind_list = ['2005 PPP conversion factor, GDP (LCU per international $)',
            'Population, female (% of total)', 
            'Ratio of female to male labor force participation rate (%) (national estimate',
            'Adjusted net national income (constant 2010 US$)',
            'Employers, female (% of female employment) (modeled ILO estimate)',
            'Expenditure on primary education (% of government expenditure on education)',
            'Law mandates nondiscrimination based on gender in hiring (1=yes; 0=no)',
            'Literacy rate, adult female (% of females ages 15 and above)',
            'Teenage mothers (% of women ages 15-19 who have had children or are currently pregnant)',
            'Women making their own informed decisions regarding sexual relations, contraceptive use and reproductive health care  (% of women age 15-49)',
            'Women who were first married by age 18 (% of women ages 20-24)',
            'Women\'s share of population ages 15+ living with HIV (%)'
           ]

less_data = data.loc[data['Country Name'].isin(geo_list) 
                     & data['Indicator Name'].isin(ind_list)]

In [37]:
less_data

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
0,Arab World,ARB,"2005 PPP conversion factor, GDP (LCU per inter...",PA.NUS.PPP.05,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,Arab World,ARB,Adjusted net national income (constant 2010 US$),NY.ADJ.NNTY.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.592444e+12,1.701558e+12,1.911251e+12,2.017596e+12,2.075619e+12,2.084663e+12,1.967601e+12,NaN,NaN,NaN
400,Arab World,ARB,"Employers, female (% of female employment) (mo...",SL.EMP.MPYR.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,1.892468e+00,1.749565e+00,1.655007e+00,1.525120e+00,1.391172e+00,1.431127e+00,1.378796e+00,1.367140,1.366209,NaN
430,Arab World,ARB,Expenditure on primary education (% of governm...,SE.XPD.PRIM.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
759,Arab World,ARB,Law mandates nondiscrimination based on gender...,SG.LAW.NODC.HR,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
773,Arab World,ARB,"Literacy rate, adult female (% of females ages...",SE.ADT.LITR.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1123,Arab World,ARB,"Population, female (% of total)",SP.POP.TOTL.FE.ZS,49.636810,49.634113,49.633242,49.633592,49.634372,49.634755,...,4.844742e+01,4.838815e+01,4.834279e+01,4.830977e+01,4.828545e+01,4.826480e+01,4.824464e+01,48.223828,NaN,NaN
1448,Arab World,ARB,Teenage mothers (% of women ages 15-19 who hav...,SP.MTR.1519.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1564,Arab World,ARB,Women making their own informed decisions rega...,SG.DMK.SRCR.FN.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1572,Arab World,ARB,Women who were first married by age 18 (% of w...,SP.M18.2024.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
